In [1]:
import sys
import os

# Get the absolute path to the project folder, then to 'utilities'
project_path = os.path.abspath("..")  # This goes up one level from 'scrapingFiles'
utilities_path = os.path.join(project_path, "utilities")
print(utilities_path)
print(sys.path)
# Add 'utilities' folder to Python's path if it's not already there
if utilities_path not in sys.path:
    sys.path.append(utilities_path)
print(project_path)


c:\Users\hp\OneDrive\Documents\redbus_scraping\redbusWebscraping\utilities
['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Program Files\\Python312\\Lib\\site-packages']
c:\Users\hp\OneDrive\Documents\redbus_scraping\redbusWebscraping


In [2]:
# Function to scrape all pages

from datetime import datetime
from commonFunctions import exportCsv, initialize_driver, load_page, scrape_bus_routes
from config import telanganaUrl
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

URL=telanganaUrl

# Function to scrape bus details
def scrape_bus_details(driver, link, name):
    try:
        #driver.get(url)
        #time.sleep(5)  # Allow the page to load
        load_page(driver,link)
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content
            # Find bus item details
            bus_name= driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name)): # Total number of bus found on the page
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {link}: {str(e)}")
        return []

def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            initializeDriver = initialize_driver()
            load_page(initializeDriver,URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(initializeDriver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                initializeDriver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                initializeDriver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(initializeDriver)
            # Iterate over each bus route link and scrape the details
            for route_link, route_name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(initializeDriver, route_link, route_name)
                if bus_details:
                    all_bus_details.extend(bus_details)

            initializeDriver.quit()
        except Exception as err:
            print(f"Error occurred while accessing page {page}: {str(err)}")

# List to hold all bus details
all_bus_details = []

# Scrape routes and details from all pages
scrape_all_pages()

# Generate a timestamp for the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") 
filename=f"Telangana_bus_details_{timestamp}.csv"
exportCsv(all_bus_details,filename)
